# Develop code to use fermipy with pointlike binning

In [1]:
import logging, yaml, glob
from uw.like2 import configuration
from skymaps import Band
from fermipy import gtanalysis

os.chdir(os.path.expandvars('$FERMI/skymodels/P305_8years/uw8210'))

In [2]:
# set up the pointlike configuration, extract stuff

cfg = configuration.Configuration('.', quiet=True, postpone=True)

os.environ['CUSTOM_IRF_NAMES']='' # it wants this set
if False:
    os.environ['CALDB']=cfg.caldb
    irf = cfg.irf
else:
    irf='P8R2_SOURCE_V6' 
print 'CALDB set to {} for irf {}'.format(os.environ['CALDB'], irf)

diffuse_dir =os.path.expandvars('$FERMI/diffuse/') 
galdiff = diffuse_dir+cfg['diffuse']['ring']['filename']
isodiff = [diffuse_dir+cfg['diffuse']['isotrop']['filename'].replace('**',x) for x in('FRONT','BACK')] 
roidir = Band(12).dir(100) # choose an arbitrary RoI
ra,dec = roidir.ra(), roidir.dec()

CALDB set to /nfs/farm/g/glast/u35/ReleaseManagerBuild/redhat6-x86_64-64bit-gcc44/Optimized/ScienceTools/11-05-00/irfs/caldb/CALDB for irf P8R2_SOURCE_V6


In [3]:
# The monthly event data and livetime cubes
lc_monthly = sorted(glob.glob('/afs/slac/g/glast/groups/catalog/P8_P302/zmax105/ltcube*.fits'))
ft1_monthly = sorted(glob.glob('/afs/slac/g/glast/groups/catalog/P8_P305/zmax105/P305*.fits'))

month=0 # use the first month

In [5]:
# set up components, here for front back on a single energy range
ie=0 #first band

order=6 #make depend on ie and evtype
components = [dict(dict(name='{:02d}'.format(et+2*ie), #gets replaced
                        selection=dict(evtype=1<<et, #evtype, 
                                       emin=10**2+ie/4., emax=10**2.25+ie/4.),
                        binning=dict(hpx_order=order), 
                        model=dict(isodiff=isodiff[et]))) 
              for et in (0,1)]
components

[{'binning': {'hpx_order': 6},
  'model': {'isodiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_FRONT_zmax80_EGBfree.txt'},
  'name': '00',
  'selection': {'emax': 177.82794100389228, 'emin': 100.0, 'evtype': 1}},
 {'binning': {'hpx_order': 6},
  'model': {'isodiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_BACK_zmax80_EGBfree.txt'},
  'name': '01',
  'selection': {'emax': 177.82794100389228, 'emin': 100.0, 'evtype': 2}}]

In [6]:
# the config dict for input to GTAnalysis
config = yaml.load("""
data:
    evfile: {ft1}
    scfile: null
    ltcube: {ltcube}
    
binning:
    projtype: 'HPX'
    enumbins: 1
    hpx_order: 6 #64 override in components
    coordsys: 'GAL'
    roiwidth : 15.0

selection:
    ra: {ra}
    dec: {dec}
    #filter: 'ZENITH_ANGLE <= 100'
    #tmin: 239557417
    #tmax: 241960000
    zmax: 100
    #evclass : 128


gtlike:
    irfs: {irf}
    
model:
    galdiff: {galdiff}
    isodiff: # different for front and back
    # what about Sun/Moon?
    #catalogs: 
    #    - '3FGL'
    sources: []

components: {components}

fileio:
    outdir: 'fermipy'

""".format(ltcube=lc_monthly[month],
           ft1=ft1_monthly[month],
           irf=irf, 
           galdiff=galdiff,
           components=components,
           ra=ra, dec=dec,
           ))
config

{'binning': {'coordsys': 'GAL',
  'enumbins': 1,
  'hpx_order': 6,
  'projtype': 'HPX',
  'roiwidth': 15.0},
 'components': [{'binning': {'hpx_order': 6},
   'model': {'isodiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_FRONT_zmax80_EGBfree.txt'},
   'name': '00',
   'selection': {'emax': 177.82794100389228, 'emin': 100.0, 'evtype': 1}},
  {'binning': {'hpx_order': 6},
   'model': {'isodiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_BACK_zmax80_EGBfree.txt'},
   'name': '01',
   'selection': {'emax': 177.82794100389228, 'emin': 100.0, 'evtype': 2}}],
 'data': {'evfile': '/afs/slac/g/glast/groups/catalog/P8_P305/zmax105/P305_Source_001_zmax105.fits',
  'ltcube': '/afs/slac/g/glast/groups/catalog/P8_P302/zmax105/ltcube_001_zmax105.fits',
  'scfile': None},
 'fileio': {'outdir': 'fermipy'},
 'gtlike': {'irfs': 'P8R2_SOURCE_V6'},
 'model': {'galdiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/g

In [7]:
reload(gtanalysis)
self = gtanalysis.GTAnalysis(config)

2018-03-29 09:22:50 INFO    GTAnalysis.__init__(): 
--------------------------------------------------------------------------------
fermipy version 0.16.0+170.gf71ab.dirty 
ScienceTools version ScienceTools-11-05-00


In [8]:
self.setup() 

2018-03-29 09:22:59 INFO    GTAnalysis.setup(): Running setup.
2018-03-29 09:22:59 INFO    GTBinnedAnalysis.setup(): Running setup for component 00
2018-03-29 09:22:59 INFO    GTBinnedAnalysis._select_data(): Start data selection files: {'ft1': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/ft1_00.fits', 'srcmdl': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/srcmdl_00.xml', 'ccube': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/ccube_00.fits', 'bexpmap': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/bexpmap_00.fits', 'ltcube': '/afs/slac/g/glast/groups/catalog/P8_P302/zmax105/ltcube_001_zmax105.fits', 'ft1_filtered': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/ft1_filtered_00.fits', 'srcmap': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/srcmap_00.fits', 'bexpmap_roi': '/nfs/farm/g/glast/g/catalog/pointlike

gtsrcmaps failed to create file /nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/srcmap_00.fits
keywords:
  minbinsz  : 0.05
  irfs      : P8R2_SOURCE_V6
  srcmdl    : /nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/srcmdl_00.xml
  emapbnds  : no
  expcube   : /afs/slac/g/glast/groups/catalog/P8_P302/zmax105/ltcube_001_zmax105.fits
  chatter   : 3
  bexpmap   : /nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/bexpmap_00.fits
  outfile   : /nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/srcmap_00.fits
  cmap      : /nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/ccube_00.fits
  evtype    : 1
  rfactor   : 2


AssertionError: 

In [16]:
!cat fermipy/fermipy.log

2018-03-29 09:22:50 INFO    GTAnalysis.__init__(): 
--------------------------------------------------------------------------------
fermipy version 0.16.0+170.gf71ab.dirty 
ScienceTools version ScienceTools-11-05-00
2018-03-29 09:22:50 DEBUG   GTAnalysis.print_config(): Configuration:
binning:
  binsperdec: 8.0
  binsz: 0.1
  coordsys: GAL
  enumbins: 1
  hpx_ebin: true
  hpx_order: 6
  hpx_ordering_scheme: RING
  npix: null
  proj: AIT
  projtype: HPX
  roiwidth: 15.0
components:
- binning:
    hpx_order: 6
  model:
    isodiff: /afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_FRONT_zmax80_EGBfree.txt
  name: '00'
  selection:
    emax: 177.82794100389228
    emin: 100.0
    evtype: 1
- binning:
    hpx_order: 6
  model:
    isodiff: /afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_BACK_zmax80_EGBfree.txt
  name: '01'
  selection:
    emax: 177.82794100389228
    emin: 100.0
    evtype: 2
data:
  cacheft1: true
  evfile: /